In [ ]:
import targets as targets
import classification as classif
import stage as stage
import rules as rules
import comms as comms
import processor as processor
import datetime

In [ ]:

ts = targets.TargetSpace()

In [ ]:
bc = classif.BackgroundClassifier()
rnc = classif.RadiusNeighborsClassifier(classif.classification_weights, ts, outliers=bc)

In [ ]:
send_triggers = rules.SendTriggers()

In [ ]:
classification_processor = processor.ClassificationProcessor(rnc, ts, send_triggers, True)

In [ ]:
stage_instance = stage.Stage(classification_processor, ts, send_triggers)

In [ ]:
comms_instance = comms.InstrumentComms(stage_instance, True)

In [ ]:
stage_instance.processEligibleStagedData()

In [ ]:
import sqlite3
conn = sqlite3.connect('current_model_targets.db')
c = conn.cursor()
c.execute('''CREATE TABLE current_model_targets 
(id,size,zpeed,deltav,target_strength,current,time_of_day,passive_acoustics,
source,date,classification)''')

In [ ]:
from config import site_name

In [ ]:
def update_classifier_database(c, stage_instance):
    """
    Enter current model values into database (with cursor c)
    """
    ident = [1] # deal with this
    site = [site_name + '_auto']

    entries = []
    for i, feat_list in enumerate(stage_instance.target_space.tables['classifier_features']):
        feats = stage_instance.target_space.tables['classifier_features'][i]
        class_placeholder = [0]
        date_placeholder = [0]
        passive_placeholder = [0]
        entry = tuple(ident + feats + passive_placeholder + site + date_placeholder + class_placeholder)
        entries.append(entry)
            
    c.executemany("INSERT INTO current_model_targets VALUES (?,?,?,?,?,?,?,?,?,?,?)", entries)

In [ ]:
update_classifier_database(c,stage_instance)